# REST ChatBot

Advantages of using SK over vanilla REST or standar OpenAI SDK:

- HttpClient handling and built-in retry logic
- Pre-defined models (Prompt & Completion)
- Process prompts as pipes
- Prompts and prompt templates (semantic functions)
- Process native functions as semantic functions


## Load the required .NET packages and supporting classes

In [2]:
using System;
using System.Net;
using System.Net.Http;
using System.Net.Http.Headers;
using System.Text;
using System.Text.Json;
using System.Text.Json.Serialization;
using dotenv.net;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

#r "nuget: dotenv.net"

#!import Models/Models.cs
#!import Utils/HttpUtils.cs

Installed Packages dotenv.net, 3.1.2

## Load the OpenAI variables from environment variables or an .env file

In [3]:
DotEnv.Load();
var uri = Environment.GetEnvironmentVariable("GPT_OPENAI_FULL_ENDPOINT");
var apiKey = Environment.GetEnvironmentVariable("GPT_OPENAI_KEY");
Console.WriteLine($"Using deployment at: {uri} with key {apiKey.Substring(0, 5)}...");

Using deployment at: https://alemorfr.openai.azure.com/openai/deployments/gpt/chat/completions?api-version=2023-03-15-preview with key 32c87...


## Get a HttpClient instance

**> Note:** This is not necessary, but it may help prevent port exhaustion issues.

In [4]:
// Note: This may be good practice depending on your use case. Otherwise, an HttpClient will be created for you.
var client = new HttpClient();
client.DefaultRequestHeaders.Accept.Add(new MediaTypeWithQualityHeaderValue("application/json"));
client.DefaultRequestHeaders.Add("api-key", $"{apiKey}");

## Call the bot for the first time

In [5]:
var DefaultSystemMessage = "You are a general assitant.";

// Keep the conversation history
var chatHistory = new List<Message>();
chatHistory.Add(new Message("system", DefaultSystemMessage));

// Get the first Chat completion
var ans = await GetGptCompletionAsync(client,uri,chatHistory);
Console.WriteLine(ans.Item1 + "\n");

As a general assistant, my role is to provide support and assistance to various departments and individuals within an organization. This may include tasks such as answering phone calls, responding to emails, scheduling appointments, organizing files, running errands, and performing other administrative duties as needed. I am responsible for ensuring that the office runs smoothly and efficiently, and that all tasks are completed in a timely and accurate manner. I am also expected to maintain a professional and courteous demeanor at all times, and to represent the organization in a positive light.



## Enter the while loop and process the user and system messages

In [6]:
while (true) {
    // Get and write the user input
    var userMessage = await InteractiveKernel.GetInputAsync("Your message");
    Console.WriteLine($"User: {userMessage}");

    switch(userMessage) {
        case "history":
            foreach (var message in chatHistory) {
                Console.WriteLine(message.Role+ ": " + message.Content);
            }
            continue;
        default:
            // send the prompt for completion, but really send the conversation
            chatHistory.Add(new Message("user", userMessage));
            ans = await GetGptCompletionAsync(client,uri,chatHistory);
            Console.WriteLine(ans.Item1 + "\n");
            
            // add the completion as a system message
            chatHistory.Add(new Message("system", ans.Item1));
            break;
    }        
}

Error: Input request cancelled

Error: Input request cancelled

Error: System.Exception: Input request cancelled
   at Microsoft.DotNet.Interactive.Kernel.GetInputAsync(String prompt, Boolean isPassword, String typeHint, String valueName) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\Kernel.Static.cs:line 71
   at Microsoft.DotNet.Interactive.Kernel.GetInputAsync(String prompt, String typeHint, String valueName) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\Kernel.Static.cs:line 46
   at Submission#9.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)